In [ ]:
print('hello')

hello


In [37]:
#Create a database for the pensioners

#Import needed packages

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

#ensure reproducibility
random.seed(1)
np.random.seed(1)

#number of pensioners :
num_pens = 100

#We need to create empty lists for each characteristic of each pensioner
pens_id = []
pens_sex = []
pens_bdate = []
pens_amount = []

#fixed initial date

initial_date = datetime(2024, 1, 1)

#loop num_pens times to create the data base

for i in range(1, num_pens+1):
    #ID
    pens_id.append(i)
    #Sex
    sex = random.choice(["Male","Female"])
    pens_sex.append(sex)
    #Birth date
    age = int(random.triangular(65, 100, 75)) #mode at 75 to have less people towards 90-100yo
    bdate = initial_date - timedelta(days=365*age + random.randint(0,365))
    pens_bdate.append(bdate.strftime("%Y-%m-%d"))
    #Pension Amount (see the last cell for how I got this distribution)
    if sex == "Male":
        pension_amount = np.random.lognormal(mean=10.748, sigma=0.5)
    else :
        pension_amount = np.random.lognormal(mean=10.35, sigma=0.5)
    pens_amount.append(pension_amount)

    
#create a DataFrame
pensioners_df = pd.DataFrame({
    'ID': pens_id,
    'Sex': pens_sex,
    'Birth Date': pens_bdate,
    'Pension Amount': pens_amount})

#convert 'Birth Date' column to datetime
pensioners_df['Birth Date'] = pd.to_datetime(pensioners_df['Birth Date'])
#round the 'Pension Amount' column to two decimals 
#to get an amount in swiss francs until cents
pensioners_df['Pension Amount'] = pensioners_df['Pension Amount'].round(2)


#print the database to check it
#print the types that I have for each variable
print(pensioners_df.head(100))
print(pensioners_df.dtypes)

     ID     Sex Birth Date  Pension Amount
0     1    Male 1943-12-20       104838.12
1     2  Female 1952-06-02        23020.07
2     3  Female 1941-10-06        24002.52
3     4    Male 1945-07-05        27214.79
4     5  Female 1943-01-20        48180.22
5     6  Female 1949-09-24         9889.52
6     7    Male 1934-01-08       111346.91
7     8    Male 1956-04-15        31805.56
8     9    Male 1931-07-13        54585.28
9    10    Male 1930-01-10        41081.57
10   11    Male 1938-05-14        96669.57
11   12    Male 1947-02-08        16612.81
12   13    Male 1938-08-27        39608.20
13   14    Male 1946-04-11        38406.15
14   15    Male 1950-08-21        82033.64
15   16    Male 1938-01-23        26850.89
16   17  Female 1944-02-12        28675.14
17   18    Male 1948-03-25        30003.54
18   19  Female 1935-05-10        31923.79
19   20  Female 1943-01-04        41831.61
20   21  Female 1949-06-27        18028.16
21   22  Female 1941-07-18        55401.49
22   23  Fe

In [38]:
#Import mortality tables
#those tables are from the Federal Office of Statistics
#https://www.bfs.admin.ch/asset/fr/30025876

import pandas as pd

Path_Male = "/files/AP_Project/px-x-0102020300_101_20240522-143851.csv"
Path_Female = "/files/AP_Project/px-x-0102020300_101_20240522-144536.csv"

df_Male = pd.read_csv(Path_Male, sep=';')
df_Female = pd.read_csv(Path_Female, sep=';')

#name each column
df_Male.columns = ['Birth Year', 'Sex', 'Age', 'Probability of Death']
df_Female.columns = ['Birth Year', 'Sex', 'Age', 'Probability of Death']

#remove the "ans" and give age as integer
df_Male['Age'] = df_Male['Age'].str.replace(' ans', '').astype(int)
df_Female['Age'] = df_Female['Age'].str.replace(' ans', '').astype(int)

#give the probability of death as a float
df_Male['Probability of Death'] = df_Male['Probability of Death'].astype(float)
df_Female['Probability of Death'] = df_Female['Probability of Death'].astype(float)

#replace "Homme" with "Male" and "Femme" with "Female"
df_Male['Sex'] = 'Male'
df_Female['Sex'] = 'Female'

#merge our two data frames
life_table_df = pd.concat([df_Male,df_Female])

print(life_table_df)

      Birth Year     Sex  Age  Probability of Death
0           1900    Male   65              0.031952
1           1900    Male   66              0.033583
2           1900    Male   67              0.036676
3           1900    Male   68              0.040693
4           1900    Male   69              0.043664
...          ...     ...  ...                   ...
7331        2030  Female  116              0.915660
7332        2030  Female  117              0.949700
7333        2030  Female  118              0.974439
7334        2030  Female  119              0.990299
7335        2030  Female  120              1.000000

[14672 rows x 4 columns]


In [39]:
#I will simulate deaths every January 1st
#First I need a function to get the age of each pensioner each January 1st

def age_function(birth_date, current_date):
    # Convert birth_date to datetime object if it's a string
    if isinstance(birth_date, str):
        birth_date = datetime.strptime(birth_date, "%Y-%m-%d")
    
    #The age is just substracting current year to the birth year 
    #minus 1 for all people not born on January 1st
    age = current_date.year - birth_date.year - 1
    #Gotta adjust the age for all the people that are born on January 1st
    if (birth_date.month == 1 and birth_date.day == 1):
        age += 1
    return age

#tests
birth_date = "2001-01-01"
current_date = datetime(2024, 1, 1)
print(age_function(birth_date, current_date))
birth_date = "2001-01-02"
current_date = datetime(2024, 1, 1)
print(age_function(birth_date, current_date))
birth_date = "2000-12-31"
current_date = datetime(2024, 1, 1)
print(age_function(birth_date, current_date))

23
22
23


In [40]:
#Now I can simulate if each pensioner dies or not comparing a random number 
#from Uniform(0,1) to the probability of death 
#(according to the pensioner sex, birth date and age)

import pandas as pd
import random

#again we need reproducibility
random.seed(1)

#Here is the function to simulate deaths for one year
def simulate_deaths(year, pensioners_df, life_table_df, deceased_ids):
    deaths = []  #store IDs of pensioners who died in the year

    #Create a dictionary for quicker lookup of mortality data
    life_table_dict = life_table_df.set_index(['Birth Year', 'Sex', 'Age']).to_dict('index')

    for index, pensioner in pensioners_df.iterrows():
        pensioner_id = pensioner['ID']
        if pensioner_id in deceased_ids:
            continue
        
        sex = pensioner['Sex']
        birth_date = pensioner['Birth Date']
        current_age = age_function(birth_date, datetime(year, 1, 1))

        #Use the dictionary for quicker lookup
        key = (birth_date.year, sex, current_age)
        if key in life_table_dict:
            probability_of_death = life_table_dict[key]['Probability of Death']
            if random.uniform(0, 1) < probability_of_death:
                deaths.append(pensioner_id)
                deceased_ids.append(pensioner_id)

    deaths_df = pd.DataFrame({'Year': [year] * len(deaths), 'ID': deaths})
    return deaths_df


#Now the function to simulate deaths over 10 years
#(to give the pension fund a 10 years projection)
def simulate_deaths_for_10_years(pensioners_df, life_table_df):
    deceased_ids = []  #To store the already deceased pensioners
    
    deaths_data = []  #To store the deaths for each year
    
    #Iterate over 10 years
    for year in range(2025, 2035):
        deaths_data.append(simulate_deaths(year, pensioners_df, life_table_df, deceased_ids))
    
    #Concatenate into a single DataFrame
    deaths_df = pd.concat(deaths_data, ignore_index=True)
    return deaths_df

#Call the function to simulate deaths over 10 years
deaths_df = simulate_deaths_for_10_years(pensioners_df, life_table_df)

#Show more rows
pd.set_option('display.max_rows', 100)

print(deaths_df)

    Year  ID
0   2025   9
1   2025  10
2   2025  14
3   2025  27
4   2025  28
5   2025  36
6   2025  72
7   2025  92
8   2026  16
9   2026  30
10  2026  31
11  2026  34
12  2026  47
13  2026  58
14  2026  61
15  2026  87
16  2026  89
17  2027  15
18  2027  40
19  2027  70
20  2027  80
21  2028   7
22  2028  39
23  2028  62
24  2028  78
25  2029  13
26  2029  35
27  2029  49
28  2029  69
29  2030  52
30  2030  54
31  2030  90
32  2031   4
33  2031  22
34  2031  50
35  2032  59
36  2033   1
37  2033   6
38  2033  48
39  2033  68
40  2033  74
41  2033  76
42  2033  82
43  2033  83
44  2033  85
45  2033  91
46  2033  93
47  2034  55
48  2034  77
49  2034  97


In [41]:
############################################################
#CHANGE HERE : Use of a dictionary for the mortality tables
############################################################

#Now I can simulate if each pensioner dies or not comparing a random number 
#from Uniform(0,1) to the probability of death 
#(according to the pensioner sex, birth date and age)

import pandas as pd
import random

#again we need reproducibility
random.seed(1)

#Here is the function to simulate deaths for one year
def simulate_deaths(year, pensioners_df, life_table_df, deceased_ids):
    deaths = []  # store IDs of pensioners who died in the year

    #create a dictionary for quick lookup of mortality data
    life_table_dict = life_table_df.set_index(['Birth Year', 'Sex', 'Age']).to_dict('index')

    for index, pensioner in pensioners_df.iterrows():
        pensioner_id = pensioner['ID']
        if pensioner_id in deceased_ids:
            continue
        
        sex = pensioner['Sex']
        birth_date = pensioner['Birth Date']
        current_age = age_function(birth_date, datetime(year, 1, 1))

        #use the dictionary for quick lookup
        key = (birth_date.year, sex, current_age)
        if key in life_table_dict:
            probability_of_death = life_table_dict[key]['Probability of Death']
            if random.uniform(0, 1) < probability_of_death:
                deaths.append(pensioner_id)
                deceased_ids.append(pensioner_id)

    deaths_df = pd.DataFrame({'Year': [year] * len(deaths), 'ID': deaths})
    return deaths_df


#Now the function to simulate deaths over 10 years
#(to give the pension fund a 10 years projection)
def simulate_deaths_for_10_years(pensioners_df, life_table_df):
    deceased_ids = []  #To store the already deceased pensioners
    
    deaths_data = []  #To store the deaths for each year
    
    #Iterate over 10 years
    for year in range(2025, 2035):
        deaths_data.append(simulate_deaths(year, pensioners_df, life_table_df, deceased_ids))
    
    #Concatenate into a single DataFrame
    deaths_df = pd.concat(deaths_data, ignore_index=True)
    return deaths_df

#Call the function to simulate deaths over 10 years
deaths_df = simulate_deaths_for_10_years(pensioners_df, life_table_df)

#Show more rows
pd.set_option('display.max_rows', 100)

print(deaths_df)

    Year  ID
0   2025   9
1   2025  10
2   2025  14
3   2025  27
4   2025  28
5   2025  36
6   2025  72
7   2025  92
8   2026  16
9   2026  30
10  2026  31
11  2026  34
12  2026  47
13  2026  58
14  2026  61
15  2026  87
16  2026  89
17  2027  15
18  2027  40
19  2027  70
20  2027  80
21  2028   7
22  2028  39
23  2028  62
24  2028  78
25  2029  13
26  2029  35
27  2029  49
28  2029  69
29  2030  52
30  2030  54
31  2030  90
32  2031   4
33  2031  22
34  2031  50
35  2032  59
36  2033   1
37  2033   6
38  2033  48
39  2033  68
40  2033  74
41  2033  76
42  2033  82
43  2033  83
44  2033  85
45  2033  91
46  2033  93
47  2034  55
48  2034  77
49  2034  97


In [42]:
print(pensioners_df['Pension Amount'].describe())

#let's compute our aggregate pension amount in 2024 January 1st
total_pension_amount = pensioners_df['Pension Amount'].sum()
print('')
print("Aggregate pension amount for 2024:", total_pension_amount)
print('')

#let's do it for 2025 to 2034
#I did a function to compute the aggregate pension amount 
#only for the remaining pensioners
def calculate_aggregate_pension(pensioners_df, deaths_df):
    #dictionary to store aggregate pension amounts for each year
    aggregate_pension = {}

    #copy of the original pensioners DataFrame
    remaining_pensioners_df = pensioners_df.copy()

    #iterate over each year in the deaths DataFrame
    for year in deaths_df['Year'].unique():
        #get the IDs of pensioners who died in the year
        deaths_ids = deaths_df[deaths_df['Year'] == year]['ID'].tolist()

        #remove deceased pensioners from the copy of pensioners DataFrame
        remaining_pensioners_df = remaining_pensioners_df[~remaining_pensioners_df['ID'].isin(deaths_ids)]

        #sum all remaining pension amounts for the year
        total_pension_amount = remaining_pensioners_df['Pension Amount'].sum()

        #store the aggregate pension amount for the year in the dictionary
        aggregate_pension[year] = total_pension_amount

    return aggregate_pension

#Call the function for aggregate pension amount
aggregate_pension = calculate_aggregate_pension(pensioners_df, deaths_df)

#I had amounts that were not rounded to 2 anymore, but it was like a 
#micro differences at 6th or 7th decimal
#I rounded the amount at 2 decimals again
for year, total_pension_amount in aggregate_pension.items():
    rounded_amount = round(total_pension_amount, 2)
    print(f"Aggregate pension amount for {year}: {rounded_amount}")

#number of deceased per year
def calculate_deaths_per_year(deaths_df):
    #dictionary to store the number of deaths per year
    deaths_per_year = {}

    #iterate over each year in the deaths DataFrame
    for year in range(2025, 2035):
        #count the number of deaths in the year
        num_deaths = deaths_df[deaths_df['Year'] == year].shape[0]
        deaths_per_year[year - 1] = num_deaths

    return deaths_per_year

#Call the function for number of deaths per year
deaths_per_year = calculate_deaths_per_year(deaths_df)

for year, num_deaths in deaths_per_year.items():
    print(f"Number of deaths in {year}: {num_deaths}")


count       100.000000
mean      45065.324900
std       22894.630561
min        9889.520000
25%       29741.565000
50%       39132.240000
75%       51593.345000
max      111346.910000
Name: Pension Amount, dtype: float64

Aggregate pension amount for 2024: 4506532.49

Aggregate pension amount for 2025: 4207518.65
Aggregate pension amount for 2026: 3846525.72
Aggregate pension amount for 2027: 3605781.52
Aggregate pension amount for 2028: 3253367.39
Aggregate pension amount for 2029: 3054071.29
Aggregate pension amount for 2030: 2920990.03
Aggregate pension amount for 2031: 2795816.67
Aggregate pension amount for 2032: 2759761.93
Aggregate pension amount for 2033: 2142318.77
Aggregate pension amount for 2034: 2048155.92
Number of deaths in 2024: 8
Number of deaths in 2025: 9
Number of deaths in 2026: 4
Number of deaths in 2027: 4
Number of deaths in 2028: 4
Number of deaths in 2029: 3
Number of deaths in 2030: 3
Number of deaths in 2031: 1
Number of deaths in 2032: 11
Number of deaths i

In [47]:
#############################################################
#Just from the change of dictionary for the mortality tables
#Monte Carlo Simulations ≈4.3sec against ≈9,5sec in my original code
#############################################################

#Monte Carlo Estimates for number of deacesed per year and aggregate pension amount
import pandas as pd
import numpy as np
import random

########################## Time for computation
import time
start_time = time.time()
##########################

#number of simulations
num_simulations = 10

#lists to store results for each simulation
total_pension_amounts_MC = []
deaths_per_year_MC = []

#loop over each simulation
base_seed = 1
for i in range(num_simulations):
    #seed for reproducibility
    random_seed = base_seed + i
    random.seed(random_seed)
    np.random.seed(random_seed)
    
    #simulate deaths for the 10 projected years
    deaths_df = simulate_deaths_for_10_years(pensioners_df, life_table_df)
    
    #aggregate pension amounts for the 10 projected years
    aggregate_pension = calculate_aggregate_pension(pensioners_df, deaths_df)
    total_pension_amounts_MC.append(aggregate_pension)
    
    #number of deaths for the 10 projected years
    deaths_per_year = calculate_deaths_per_year(deaths_df)
    deaths_per_year_MC.append(deaths_per_year)

#DataFrame for results
total_pension_amounts_df = pd.DataFrame(total_pension_amounts_MC)
deaths_per_year_df = pd.DataFrame(deaths_per_year_MC)

#transpose the DataFrames so that simulations are columns and years are rows
total_pension_amounts_df = total_pension_amounts_df.T
deaths_per_year_df = deaths_per_year_df.T

#name the columns to indicate the simulation number
total_pension_amounts_df.columns = [f"Simulation{i+1}" for i in range(num_simulations)]
deaths_per_year_df.columns = [f"Simulation{i+1}" for i in range(num_simulations)]

#Again, I look at the people that died in 2024 on January 1st 2025
#So the death in 2024 are taken into account in 2025 and so on
#Hence I do
deaths_per_year_df.index = deaths_per_year_df.index - 1

print("Total Pension Amounts DataFrame:")
print(total_pension_amounts_df)
print('')
print("Number of Deaths DataFrame:")
print(deaths_per_year_df)

#Compute the Monte Carlo estimates as the mean of the randomly 
#simulated amounts
mean_pension_amounts = total_pension_amounts_df.mean(axis=1)
mean_deaths_per_year = deaths_per_year_df.mean(axis=1)

print("Monte Carlo Estimates - Expected Aggregate Pension Amounts:")
print(mean_pension_amounts)
print('')
print("Monte Carlo Estimates - Expected Number of Deaths per Year:")
print(mean_deaths_per_year)

################################### Time for computation
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for {num_simulations} simulations")
####################################

Total Pension Amounts DataFrame:
      Simulation1  Simulation2  Simulation3  Simulation4  Simulation5  \
2025   4207518.65   4451131.00   4256870.36   4352134.74   4109138.49   
2026   3846525.72   3842964.03   4107933.17   4077550.62   3968557.37   
2027   3605781.52   3615431.95   3644608.65   3797788.76   3561813.79   
2028   3253367.39   3413303.97   3447925.51   3443695.24   3384169.24   
2029   3054071.29   3331879.40   3270617.30   3344521.39   2981357.73   
2030   2920990.03   3227521.26   3063214.87   3002466.66   2781697.69   
2031   2795816.67   2885606.75   3000206.35   2860556.66   2566052.34   
2032   2759761.93   2780603.13   2662531.92   2603131.18   2100405.07   
2033   2142318.77   2461686.60   2590087.81   2474942.95   1967723.11   
2034   2048155.92   2257744.66   2401050.84   2139644.89   1866052.61   

      Simulation6  Simulation7  Simulation8  Simulation9  Simulation10  
2025   4235301.17   4151156.15   4234946.82   4177771.30    4327208.91  
2026   4119769.55

In [57]:
#Parallelize

#Too much of a change to apply the Numba we saw in the lecture
#Chat GPT helped me finding another package that was easy to implement
#Again it is much faster to compute here than in my
#original code here ≈10secs for 100 simulations
#against ≈9,5sec for only 10 simulations in my original code


########################## Time for computation
import time
start_time = time.time()
##########################

from joblib import Parallel, delayed
import pandas as pd
import numpy as np  # Added for random seed

# Define a function to perform the Monte Carlo simulation for a single simulation
def simulate_single_simulation(pensioners_df, life_table_df, seed):  # Added 'seed' parameter
    
    random.seed(seed)
    np.random.seed(seed)  # Set seed for this simulation
    
    # Simulate deaths for the 10 projected years
    deaths_df = simulate_deaths_for_10_years(pensioners_df, life_table_df)
    
    # Calculate aggregate pension amounts for the 10 projected years
    aggregate_pension = calculate_aggregate_pension(pensioners_df, deaths_df)
    
    # Calculate number of deaths for the 10 projected years
    deaths_per_year = calculate_deaths_per_year(deaths_df)
    
    return aggregate_pension, deaths_per_year

# Number of simulations
num_simulations = 100

# Parallelize the simulations
results = Parallel(n_jobs=-1)(delayed(simulate_single_simulation)(
    pensioners_df, life_table_df, seed) for seed in range(1, num_simulations + 1))

# Separate the results into aggregate pension amounts and deaths per year
aggregate_pension_results, deaths_per_year_results = zip(*results)

# Convert the results into DataFrames
total_pension_amounts_df = pd.DataFrame(aggregate_pension_results).T
deaths_per_year_df = pd.DataFrame(deaths_per_year_results).T

# Name the columns to indicate the simulation number
total_pension_amounts_df.columns = [f"Simulation_{i+1}" for i in range(num_simulations)]
deaths_per_year_df.columns = [f"Simulation_{i+1}" for i in range(num_simulations)]

# Print the DataFrames
print("Total Pension Amounts DataFrame:")
print(total_pension_amounts_df)

print("\nNumber of Deaths DataFrame:")
print(deaths_per_year_df)

# Monte Carlo Estimates - Mean Pension Amounts and Mean Number of Deaths per Year
mean_pension_amounts = total_pension_amounts_df.mean(axis=1).round(2)
mean_deaths_per_year = deaths_per_year_df.mean(axis=1)

print("\nMonte Carlo Estimates - Expected Aggregate Pension Amounts:")
print(mean_pension_amounts)

print("\nMonte Carlo Estimates - Expected Number of Deaths per Year:")
print(mean_deaths_per_year)

################################### Time for computation
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds for {num_simulations} simulations")
####################################

Total Pension Amounts DataFrame:
      Simulation_1  Simulation_2  Simulation_3  Simulation_4  Simulation_5  \
2025    4207518.65    4451131.00    4256870.36    4352134.74    4109138.49   
2026    3846525.72    3842964.03    4107933.17    4077550.62    3968557.37   
2027    3605781.52    3615431.95    3644608.65    3797788.76    3561813.79   
2028    3253367.39    3413303.97    3447925.51    3443695.24    3384169.24   
2029    3054071.29    3331879.40    3270617.30    3344521.39    2981357.73   
2030    2920990.03    3227521.26    3063214.87    3002466.66    2781697.69   
2031    2795816.67    2885606.75    3000206.35    2860556.66    2566052.34   
2032    2759761.93    2780603.13    2662531.92    2603131.18    2100405.07   
2033    2142318.77    2461686.60    2590087.81    2474942.95    1967723.11   
2034    2048155.92    2257744.66    2401050.84    2139644.89    1866052.61   

      Simulation_6  Simulation_7  Simulation_8  Simulation_9  Simulation_10  \
2025    4235301.17    4151156

In [58]:
#import matplotlib.pyplot as plt

#plot and export the expected pension amounts over the years
#plt.figure(figsize=(10, 6))
#plt.plot(mean_pension_amounts, marker='o', linestyle='-')
#plt.title("Monte Carlo Estimates - Expected Aggregate Pension Amounts")
#plt.xlabel("Year")
#plt.ylabel("Expected Aggregate Pension Amounts (in Millions)")
#plt.grid(True)
#plt.savefig("/files/AP_Project/Plot_amounts.jpeg", format='jpeg')
#plt.close()

#export Monte Carlo estimates
#mean_pension_amounts.to_csv("/files/AP_Project/MCamounts.csv", index_label="Year")
#mean_deaths_per_year.to_csv("/files/AP_Project/MCdeaths.csv", index_label="Year")